In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

positive_df = pd.read_csv('positive_dataset_2.csv')
negative_df = pd.read_csv('negative_dataset_2.csv')

# Standardize column names
positive_df = positive_df.rename(columns={'1sec_chunk': '1sec_chunk'})
negative_df = negative_df.rename(columns={'1secondchunk': '1sec_chunk_no_buzz',})

# Subtract 19 from each value in the '1sec_chunk_no_buzz' column of the negative_df
negative_df['1sec_chunk_no_buzz'] = negative_df['1sec_chunk_no_buzz'] - 19
# Rename '1sec_chunk_no_buzz' to '1sec_chunk'
negative_df = negative_df.rename(columns={'1sec_chunk_no_buzz': '1sec_chunk'})
positive_df['1sec_chunk'] = positive_df['1sec_chunk'].fillna(0)
negative_df['1sec_chunk'] = negative_df['1sec_chunk'].fillna(0)
if '19' not in positive_df.columns:
    positive_df['19'] = 0

positive_df['label'] = 1  # Buzz present
negative_df['label'] = 0  # No buzz

combined_df = pd.concat([positive_df, negative_df], ignore_index=True)
print("Data Loaded and Combined")
print(combined_df.head())
feature_columns = [str(i) for i in range(19)] + ['1sec_chunk','MAX_count', 'sum']
X = combined_df[feature_columns].values
y = combined_df['label'].values
print("Features and Labels Extracted")
print(f"Feature shape: {X.shape}, Label shape: {y.shape}")
combined_df.fillna(0, inplace=True)
X = combined_df[feature_columns].values
y = combined_df['label'].values
print("Missing values handled")
print(f"Feature shape: {X.shape}, Label shape: {y.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Data Split into Training and Testing Sets")
print(f"Training set size: {X_train.shape[0]}, Testing set size: {X_test.shape[0]}")

Data Loaded and Combined
                                     Input.audio_url buzz_majority  buzz_mean  \
0  https://patrickchwalek.com/audio/native/mp3/Na...           yes   1.000000   
1  https://patrickchwalek.com/audio/native/mp3/Na...           yes   0.666667   
2  https://patrickchwalek.com/audio/invasive/mp3/...           yes   0.666667   
3  https://patrickchwalek.com/audio/native/mp3/Na...            no   0.333333   
4  https://patrickchwalek.com/audio/invasive/mp3/...           yes   1.000000   

   0  1  2  3  4  5  6  ...  14  15  16  17  18  sum  MAX_count  1sec_chunk  \
0  0  0  0  3  0  0  0  ...   3   0   0   0   0   13          4           1   
1  0  0  0  0  0  0  0  ...   0   0   0   0   0   13          4           1   
2  0  0  0  1  0  0  0  ...   3   0   0   0   2   13          4           1   
3  0  0  2  1  0  0  1  ...   0   0   0   0   0   13          4           1   
4  4  0  0  0  0  0  0  ...   0   0   0   0   0   13          4           1   

    19  label

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = nn.ConstantPad1d((0, -padding), 0) #trim the padding 
        self.relu1 = nn.ReLU() #could  consdier using sigmoid or a diffrent activation function
        self.dropout1 = nn.Dropout(dropout) #sets random fraction of smaple to 0 for avioding overfitting
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None #match the dimensions of I/O if they are different, enabling residual connections
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01) #followed a normal distribution for initializing weights of  conv and downsample layer
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x): #pass on temporal block
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)
        
#define architecture of the temporal layer
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)  #number of  temporal blocks
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,  #append new block to layer array
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]
        self.network = nn.Sequential(*layers)

    def forward(self, x):   
        return self.network(x)

#add a linear layer while using temproal block cearted before
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, x):
        y1 = self.tcn(x)
        return self.linear(y1[:, :, -1])

print("TCN Model Defined")

TCN Model Defined


In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) #we could heighten the batch size based on our  compute resource
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print("Data Prepared for PyTorch")
print(f"Train loader length: {len(train_loader)}, Test loader length: {len(test_loader)}")

Data Prepared for PyTorch
Train loader length: 61, Test loader length: 16


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TCN(input_size=1, output_size=1, num_channels=[25, 25], kernel_size=7, dropout=0.3).to(device) #fine tuning can be done here
criterion = nn.BCEWithLogitsLoss() #suitable for binary classification tasks where the output is a probability, predict vs actual binaires
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5) #common optimizers for bianry classification problems and parameters, could finetune later

num_epochs = 20 
best_val_loss = float('inf')
early_stop_patience = 5
early_stop_counter = 0

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device) #this might be where Marco in terms of device setting(not for  training will  use cluster).
        optimizer.zero_grad() #clear graduates from tensors
        outputs = model(X_batch)
        loss = criterion(outputs.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() 
    epoch_loss /= len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')


Epoch 1/20, Loss: 0.2856
Epoch 2/20, Loss: 0.0934
Epoch 3/20, Loss: 0.0468
Epoch 4/20, Loss: 0.0212
Epoch 5/20, Loss: 0.0174
Epoch 6/20, Loss: 0.0123
Epoch 7/20, Loss: 0.0073
Epoch 8/20, Loss: 0.0071
Epoch 9/20, Loss: 0.0054
Epoch 10/20, Loss: 0.0047
Epoch 11/20, Loss: 0.0049
Epoch 12/20, Loss: 0.0048
Epoch 13/20, Loss: 0.0055
Epoch 14/20, Loss: 0.0041
Epoch 15/20, Loss: 0.0029
Epoch 16/20, Loss: 0.0028
Epoch 17/20, Loss: 0.0033
Epoch 18/20, Loss: 0.0021
Epoch 19/20, Loss: 0.0024
Epoch 20/20, Loss: 0.0017


In [6]:
# Validation
model.eval()
val_loss = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs.squeeze(), y_batch)
        val_loss += loss.item()
val_loss /= len(test_loader)
print(f'Validation Loss: {val_loss:.4f}')


# Early Stopping for overfitting and generalization
if val_loss < best_val_loss:
    best_val_loss = val_loss
    early_stop_counter = 0
else:
    early_stop_counter += 1
    if early_stop_counter >= early_stop_patience:
        print("Early stopping")

print("Model Training Completed")

Validation Loss: 0.0001
Model Training Completed


In [7]:
# Final evaluation on the test set
model.eval()
test_loss = 0
correct = 0
with torch.no_grad(): #diabsles gradient computation to save memory, may not be needed
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device) #X_batch contains input data, y_batch contains true labels.
        outputs = model(X_batch)
        test_loss += criterion(outputs.squeeze(), y_batch).item()
        predicted = (outputs.squeeze() > 0.5).float()
        correct += (predicted == y_batch).sum().item()

test_loss /= len(test_loader.dataset)
accuracy = correct / len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')

Test Loss: 0.0000, Accuracy: 1.0000


In [8]:
#haver to see what cross validation is being done

# from sklearn.model_selection import KFold
# import numpy as np

# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# cv_accuracies = []

# for train_index, val_index in kf.split(X):
#     X_train_cv, X_val_cv = X[train_index], X[val_index]
#     y_train_cv, y_val_cv = y[train_index], y[val_index]

#     X_train_tensor = torch.tensor(X_train_cv, dtype=torch.float32).unsqueeze(1)
#     y_train_tensor = torch.tensor(y_train_cv, dtype=torch.float32)

#     X_val_tensor = torch.tensor(X_val_cv, dtype=torch.float32).unsqueeze(1)
#     y_val_tensor = torch.tensor(y_val_cv, dtype=torch.float32)

#     train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
#     val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

#     train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

#     model = TCN(input_size=1, output_size=1, num_channels=[25, 25], kernel_size=7, dropout=0.2).to(device)
#     criterion = nn.BCEWithLogitsLoss()
#     optimizer = optim.Adam(model.parameters(), lr=0.001)

#     num_epochs = 20

#     for epoch in range(num_epochs):
#         model.train()
#         for X_batch, y_batch in train_loader:
#             X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#             optimizer.zero_grad()
#             outputs = model(X_batch)
#             loss = criterion(outputs.squeeze(), y_batch)
#             loss.backward()
#             optimizer.step()

#     model.eval()
#     correct = 0
#     with torch.no_grad():
#         for X_batch, y_batch in val_loader:
#             X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#             outputs = model(X_batch)
#             predicted = (outputs.squeeze() > 0.5).float()
#             correct += (predicted == y_batch).sum().item()

#     accuracy = correct / len(val_loader.dataset)
#     cv_accuracies.append(accuracy)
#     print(f'Fold Accuracy: {accuracy:.4f}')

# mean_cv_accuracy = np.mean(cv_accuracies)
# print(f'Mean Cross-Validation Accuracy: {mean_cv_accuracy:.4f}')


Fold Accuracy: 1.0000
Fold Accuracy: 1.0000
Fold Accuracy: 1.0000
Fold Accuracy: 1.0000
Fold Accuracy: 1.0000
Mean Cross-Validation Accuracy: 1.0000
